In [1]:
# Import the basic spark library

from pyspark.sql import SparkSession

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("schemaBook") \
      .getOrCreate()
# master contains the URL of your remote spark instance or 'local'

22/12/07 02:13:52 WARN Utils: Your hostname, MacBook-Pro-di-Cristina.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
22/12/07 02:13:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/07 02:13:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/07 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/07 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/07 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/12/07 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/12/07 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/12/07 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/12/07 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/12/07 02:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.


In [2]:
#schema book
df_book = spark.read.option("multiline","true").json("book-db-manual.json")
df_book.printSchema()
#df_book.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- crossref: string (nullable = true)
 |-- editor: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isbn: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- publisher: string (nullable = true)
 |-- series: struct (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- 

In [3]:
#schema article
df_article = spark.read.option("multiline","true").json("article-db-manual.json")
df_article.printSchema()
#df_article.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- cite: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- journal: string (nullable = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- publisher: string (nullable = true)
 |-- title: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- year: long (nullable = true)



In [4]:
#schema incollection
df_incollection = spark.read.option("multiline","true").json("incollection-db-manual.json")
df_incollection.printSchema()
#df_incollection.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- crossref: string (nullable = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [5]:
#schema www
df_www = spark.read.option("multiline","true").json("www-db-manual.json")
df_www.printSchema()
#df_www.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- key: string (nullable = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)



1) VERSION 1: Best PROFESSORS according to number of citations at a given university

The following query can be used to find the authors affiliated with a specific university
(e.g.’University of Malaga, Spain’) ordering them by the average number of citations across their
publications(only incollections contained in collections, which can be books or articles).

In [6]:
#import
from pyspark.sql.functions import col                       # Filtering using the col() function
from pyspark.sql.functions import array_contains            # Filtering on array columns
from pyspark.sql.functions import explode                   # Explode Arrays in Individual Rows
from pyspark.sql.functions import avg                       # Multiple Aggregations



In [7]:
selected_df_book = df_book  \
    .select(df_book.key)
#exploded_df_book.printSchema()
#exploded_df_book.show(truncate=False)


In [8]:
selected_df_article = df_article  \
    .select(df_article.key)
#exploded_df_article.printSchema()
#exploded_df_article.show(truncate=False)

In [9]:
exploded_df_incollection = df_incollection  \
    .select(explode(df_incollection.author), df_incollection.citations, df_incollection.crossref )    \
    .withColumnRenamed("col", "author")
#exploded_df_incollection.printSchema()
#exploded_df_incollection.show(truncate=False)

In [10]:
filtered_df_www=df_www.filter(array_contains(df_www.note, "University of Malaga, Department of Computer Sciences, Spain")).select(explode(df_www.author)).withColumnRenamed("col", "author")
#filtered_df_www.printSchema()
#filtered_df_www.show(truncate=False)

In [11]:
df_pub= selected_df_article.unionByName(selected_df_book, allowMissingColumns=True)
#df_pub.show(truncate=False)

In [12]:
df_pub=exploded_df_incollection.join(df_pub, exploded_df_incollection.crossref == df_pub.key, "left").select(exploded_df_incollection.author,exploded_df_incollection.citations)
#df_pub.show(truncate=False)

In [13]:
result = filtered_df_www.join(df_pub, filtered_df_www.author == df_pub.author, "left").drop(df_pub.author).groupBy("author").agg(
    avg("citations").alias("Avg Cit")).filter(col("Avg Cit")>="30").select(col("author.name"), col("Avg Cit")).sort(col("Avg Cit").desc())
result.show(truncate=False)

+-----------------+-------+
|name             |Avg Cit|
+-----------------+-------+
|Enrique Alba 0001|115.0  |
+-----------------+-------+

